# Histology Dataset attempt one

In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.color import rgb2gray

import torch
from torch import nn
import kornia as K
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
import timm

import import_ipynb 
import dl_helper
print(dl_helper.DEVICE)

cuda


In [2]:
def read_histo_from_path(path):
    print(f"read from {path}")
    names = []
    labels = []
    for i in os.listdir(path):
        tmp = os.listdir(path+'/'+i)
        names.extend(tmp)
        labels.extend([i] * len(tmp))
    data = []
    new_labels = []
    for i, name in enumerate(names):
        print(f"{i+1}/{len(names)}      ", end="\r")
        read = Image.open(f"{path}/{labels[i]}/{name}")
        if read.size == (50, 50) and read.mode == 'RGB':
            # data.append(rgb2gray(read))
            tmp = TF.to_tensor(read)
            data.append(tmp.reshape(1, *tmp.shape))
            new_labels.append(int(labels[i][0]))
    data = torch.cat(data)
    labels = torch.tensor(new_labels)
    print('')
    return data, labels

def read_img(path='/train'):
    path = "../data/Breast Histopathology Images SMALL"+path
    x, y = read_histo_from_path(path)
    # return reshape(x), y
    return x, y


def read_histo_small():
    path = "../data/Breast Histopathology Images SMALL"
    train_path = path+'/train'
    val_path = path+'/valid'
    test_path = path+'/test'
    x_train, y_train = read_histo_from_path(train_path)
    x_val, y_val = read_histo_from_path(val_path)
    x_test, y_test = read_histo_from_path(test_path)
    # return reshape(x_train), y_train, reshape(x_val), y_val,reshape(x_test), y_test
    return x_train, y_train, x_val, y_val, x_test, y_test

def reshape(x):
    return x.permute(0,3,2,1)

In [3]:
x_train, y_train = read_img('/train') 
x_val, y_val = read_img('/valid')
x_test, y_test = read_img('/test')

read from ../data/Breast Histopathology Images SMALL/train


KeyboardInterrupt: 

In [ ]:
plt.imshow(x_train[0].permute(1,2,0), cmap='magma')

In [ ]:
x_train.shape

In [ ]:
# Taken from here https://stackoverflow.com/a/58748125/1983544
num_workers = os.cpu_count() 
if 'sched_getaffinity' in dir(os):
    num_workers = len(os.sched_getaffinity(0))
print (num_workers)

In [ ]:
batch_size = 32
train_dataloader = DataLoader(list(zip(x_train, y_train)),
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers = num_workers)
valid_dataloader = DataLoader(list(zip(x_val, y_val)),
                              batch_size=batch_size,
                              shuffle=False,
                              drop_last=False,
                              num_workers = num_workers)
test_dataloader = DataLoader(x_test,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False,
                             num_workers = num_workers)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
lambda_l2 = 1e-4
EPOCHS = 4

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Resnet18

### Not pre-trained

In [ ]:
model_18_ut = timm.create_model('resnet18', pretrained=False, num_classes=2)
print(model_18_ut.fc)

In [ ]:
optimizer = torch.optim.Adam(model_18_ut.fc.parameters(),
                             lr=learning_rate,
                             eps=1e-2,
                             weight_decay=lambda_l2)  # built-in L2

epochs = EPOCHS

In [ ]:
model_18_ut = dl_helper.train_epochs(train_dataloader, valid_dataloader, model_18_ut, loss_fn, optimizer, epochs=epochs, log=1)
res_18_ut = dl_helper.test(test_dataloader, model_18_ut)
acc_18_ut = dl_helper.evaluate_pred(y_test, res_18_ut)
print('acc', acc_18_ut)

### Pre-trained

In [ ]:
model_18_pt = timm.create_model('resnet18', pretrained=True, num_classes=2)
print(model_18_pt.fc)

In [ ]:
optimizer = torch.optim.Adam(model_18_pt.fc.parameters(),
                             lr=learning_rate,
                             eps=1e-2,
                             weight_decay=lambda_l2)  # built-in L2

epochs = EPOCHS

In [ ]:
model_18_pt = dl_helper.train_epochs(train_dataloader, valid_dataloader, model_18_pt, loss_fn, optimizer, epochs=epochs, log=1)
res_18_pt = dl_helper.test(test_dataloader, model_18_pt)
acc_18_pt = dl_helper.evaluate_pred(y_test, res_18_pt)
print('acc', acc_18_pt)

## Resnet50

### Not pre-trained

In [ ]:
model_50_ut = timm.create_model('resnet50', pretrained=False, num_classes=2)
print(model_50_ut.fc)

In [ ]:
optimizer = torch.optim.Adam(model_50_ut.fc.parameters(),
                             lr=learning_rate,
                             eps=1e-2,
                             weight_decay=lambda_l2)  # built-in L2

epochs = EPOCHS

In [ ]:
model_50_ut = dl_helper.train_epochs(train_dataloader, valid_dataloader, model_50_ut, loss_fn, optimizer, epochs=epochs, log=1)
res_50_ut = dl_helper.test(test_dataloader, model_50_ut)
acc_50_ut = dl_helper.evaluate_pred(y_test, res_50_ut)
print('acc', acc_50_ut)

### Pre-trained

In [ ]:
model_50_pt = timm.create_model('resnet50', pretrained=True, num_classes=2)
print(model_50_pt.fc)

In [ ]:
optimizer = torch.optim.Adam(model_50_pt.fc.parameters(),
                             lr=learning_rate,
                             eps=1e-2,
                             weight_decay=lambda_l2)  # built-in L2

epochs = EPOCHS

In [ ]:
model_50_pt = dl_helper.train_epochs(train_dataloader, valid_dataloader, model_50_pt, loss_fn, optimizer, epochs=epochs, log=1)
res_50_pt = dl_helper.test(test_dataloader, model_50_pt)
acc_50_pt = dl_helper.evaluate_pred(y_test, res_50_pt)
print('acc', acc_50_pt)

## Results

In [ ]:
print('Accuracy resnet_18 not pretrained :', acc_18_ut)
print('Accuracy resnet_18 pretrained     :', acc_18_pt)
print('Accuracy resnet_50 not pretrained :', acc_50_ut)
print('Accuracy resnet_50 pretrained     :', acc_50_pt)